In [1]:
import pandas as pd

In [2]:
# %time df = pd.read_csv('large_dataset.csv')
# print(df.shape)
# df.head()

In [3]:
import vaex

In [4]:
# vaex_df = vaex.from_pandas(df)
# print(type(vaex_df))
# vaex_df.export_hdf5('large_data.hdf5')

download dataset [here](https://drive.google.com/file/d/1U__WYRE-TWFOtHGOYyXQHwzdw-VMrwgu/view)

In [5]:
%time vaex_df = vaex.open('large_data.hdf5')

CPU times: total: 484 ms
Wall time: 516 ms


It just took 484 ms to load 1.1 GB dataset, amazing right.

In [6]:
vaex_df.head()

#,f1,f2,f3,f4,target
0,0.203936,-0.291191,-0.617251,-1.4613,10.7967
1,-1.19543,0.0692141,0.291861,1.97251,11.154
2,-2.56027,-0.827846,0.405551,-0.132588,-20.3427
3,-1.1312,0.252654,0.961175,1.23687,1.40764
4,0.858774,-0.461605,1.0955,-0.87158,-17.0431
5,0.63273,2.07437,-0.700978,1.36085,49.2754
6,-1.92771,0.155743,-0.394158,-1.3712,11.0902
7,-0.165462,-0.481113,-1.29813,1.75204,-41.1136
8,-1.31883,0.393728,-1.83311,-0.599728,1.87724
9,-0.671377,-0.172073,-0.75585,1.05745,-26.2623


In [7]:
vaex_df = vaex_df.shuffle()
df_train, df_test = vaex_df.ml.train_test_split(test_size=0.2)

C:\Users\thaku\anaconda3\envs\deep-learning\lib\site-packages\vaex\ml\__init__.py:28: UserWarning: Make sure the DataFrame is shuffled
  warnings.warn('Make sure the DataFrame is shuffled')


In [8]:
df_train.shape

(24000000, 5)

In [9]:
df_test.shape

(6000000, 5)

In [10]:
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDRegressor

In [11]:
features = ['f1','f2','f3','f4']
target = 'target'

In [12]:
model = SGDRegressor()

In [13]:
vaex_model = IncrementalPredictor(features=features, target=target, model=model, batch_size=500000)
vaex_model.fit(df=df_train, progress='widget')

In [14]:
df_test = vaex_model.transform(df_test)

In [15]:
df_test.head()

#,f1,f2,f3,f4,target,prediction
0,-0.471503,-1.16984,-3.43323,-0.108833,-31.4137,-27.5899
1,-0.319622,0.694952,0.145402,-0.617327,-10.7958,16.5934
2,1.11763,1.3145,0.91143,-0.564424,43.1347,31.6443
3,1.15673,-0.00602041,0.951329,0.303354,39.3848,-0.215818
4,-0.861004,-0.289867,-2.53523,-0.989466,0.0405576,-6.6412
5,-0.0385882,-0.259006,2.44187,0.0960686,40.279,-6.89754
6,1.14564,0.964074,0.534795,0.695921,52.4746,23.2691
7,-0.353849,-0.273484,1.52172,0.505745,-27.0124,-7.10223
8,1.25037,1.45035,-1.53682,-0.395813,63.0019,35.5107
9,1.35137,0.51176,0.0204803,-1.3794,8.64062,12.5475


In [16]:
from sklearn.metrics import r2_score, mean_absolute_error

In [17]:
print(r2_score(df_test['target'].values, df_test['prediction'].values))
print(mean_absolute_error(df_test['target'].values, df_test['prediction'].values))

0.5917139579230364
15.970253181052126
